# **Mobile Game Company's Rolling Retention Analysis**

In [ ]:
#set GOOGLE_APPLICATION_CREDENTIALS=KEY_PATH
##keypath is the json key of service account
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/kashi/Desktop/JUNO/SQL/SQL Study/junobigqueryclass-key.json"

In [ ]:
%load_ext google.cloud.bigquery

# Item Info table

In [ ]:
%%bigquery
SELECT *
FROM
  `junobigqueryclass.game_data.item_info`
LIMIT
  1

# Matches Info table

In [ ]:
%%bigquery
SELECT *
FROM `junobigqueryclass.game_data.matches_info` 


# Player Info table

In [ ]:
%%bigquery
SELECT *
FROM `junobigqueryclass.game_data.player_info` 


# Purchase Info table

In [ ]:
%%bigquery
SELECT *
FROM `junobigqueryclass.game_data.purchase_info` 
LIMIT 1

## **Finding Player Joined and Retained players on a particular day**

#### 1. The total number of players who joined on a given day

In [ ]:
%%bigquery
SELECT joined as joining_day, 
COUNT(player_id) players_joined
FROM `junobigqueryclass.game_data.player_info` 
GROUP BY 1
ORDER BY 1

#### 2. Last play day table

In [ ]:
%%bigquery
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1

#### 3. Joining lastplay table for each player 

In [ ]:
%%bigquery
SELECT players.player_id,
    players.joined as joining_day,
    lastplay.lastplay_day
FROM `junobigqueryclass.game_data.player_info` players
JOIN 
(
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1
) as lastplay

ON players.player_id = lastplay.player_lastday

#### 4. Getting the list of retained player from above table
##### condition : if a given player play a match 30 days after he or she joined is a retained player

In [ ]:
%%bigquery
SELECT days_info.joining_day,
        days_info.player_id as player_id,
        CASE 
        WHEN days_info.lastplay_day - days_info.joining_day > 30 THEN 1
        ELSE 0
        END AS retained
FROM
(SELECT players.player_id,
    players.joined as joining_day,
    lastplay.lastplay_day
FROM `junobigqueryclass.game_data.player_info` players
JOIN 
(
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1
) as lastplay

ON players.player_id = lastplay.player_lastday) as days_info
ORDER BY 1



#### 5.  Aggregating over Joining_Day to get total players joined vs total players retained

In [ ]:
%%bigquery

SELECT DISTINCT joining_day as day, 
    COUNT(player_id) OVER (PARTITION BY joining_day) players_joined,
    SUM(retained) OVER (PARTITION BY joining_day) AS players_retained,

FROM
(
SELECT days_info.joining_day,
        days_info.player_id as player_id,
        CASE 
        WHEN days_info.lastplay_day - days_info.joining_day > 30 THEN 1
        ELSE 0
        END AS retained
FROM
(SELECT players.player_id,
    players.joined as joining_day,
    lastplay.lastplay_day
FROM `junobigqueryclass.game_data.player_info` players
JOIN 
(
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1
) as lastplay

ON players.player_id = lastplay.player_lastday) as days_info
ORDER BY 1
) AS t2
ORDER BY 1


# Fractional Retention
#### *(players_retained divided by players_joined)*

#### 6. Dividing column third by second to get fractional retention

In [ ]:
%%bigquery
SELECT 
  DISTINCT joining_day as day, 
  COUNT(player_id) OVER (PARTITION BY joining_day) players_joined, 
  SUM(retained) OVER (PARTITION BY joining_day) AS players_retained, 
  SUM(retained) OVER (PARTITION BY joining_day) / COUNT(player_id) OVER (PARTITION BY joining_day) fractional_retention 
FROM 
  (
    SELECT 
      days_info.joining_day, 
      days_info.player_id as player_id, 
      CASE WHEN days_info.lastplay_day - days_info.joining_day > 30 THEN 1 ELSE 0 END AS retained 
    FROM 
      (
        SELECT 
          players.player_id, 
          players.joined as joining_day, 
          lastplay.lastplay_day 
        FROM 
          `junobigqueryclass.game_data.player_info` players 
          JOIN (
            SELECT 
              player_id as player_lastday, 
              MAX(day) as lastplay_day 
            FROM 
              `junobigqueryclass.game_data.matches_info` 
            GROUP BY 
              1
          ) as lastplay ON players.player_id = lastplay.player_lastday
      ) as days_info
  ) 
ORDER BY 
  1


# Do players with rolling 30-day retention spend more?


#### 7. Joining Purchase_info table and item_info table to get money spent by each player

In [ ]:
%%bigquery
SELECT player_id,
       SUM(price)
FROM `junobigqueryclass.game_data.item_info` i
JOIN `junobigqueryclass.game_data.purchase_info` p
ON i.item_id = p.item_id
GROUP BY 1

#### 8.  Using the same logic from cell In[5] above to get retained and unretained players and joining with the table above

In [ ]:
%%bigquery
SELECT *
FROM
(SELECT player_id as pid,
       SUM(price) as amount_spent
FROM `junobigqueryclass.game_data.item_info` i
JOIN `junobigqueryclass.game_data.purchase_info` p
ON i.item_id = p.item_id
GROUP BY 1) AS player_spent

JOIN
(SELECT days_info.player_id as player_id,
        CASE 
        WHEN days_info.lastplay_day - days_info.joining_day > 30 THEN "retained"
        ELSE 'unretained'
        END AS player_status
FROM
(SELECT players.player_id,
    players.joined as joining_day,
    lastplay.lastplay_day
FROM `junobigqueryclass.game_data.player_info` players
JOIN 
(
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1
) as lastplay

ON players.player_id = lastplay.player_lastday) as days_info) as player_status

ON player_status.player_id = player_spent.pid

#### 9. Finally we aggregate the data over the `player_status` to find the sum of the amount spent by retained and unretained players and compare it.

In [ ]:
%%bigquery
SELECT player_status,
    SUM(amount_spent) as total_spent
FROM

(SELECT *
FROM
(SELECT player_id as pid,
       SUM(price) as amount_spent
FROM `junobigqueryclass.game_data.item_info` i
JOIN `junobigqueryclass.game_data.purchase_info` p
ON i.item_id = p.item_id
GROUP BY 1) AS player_spent

JOIN
(SELECT days_info.player_id as player_id,
        CASE 
        WHEN days_info.lastplay_day - days_info.joining_day > 30 THEN "retained"
        ELSE 'unretained'
        END AS player_status
FROM
(SELECT players.player_id,
    players.joined as joining_day,
    lastplay.lastplay_day
FROM `junobigqueryclass.game_data.player_info` players
JOIN 
(
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1
) as lastplay

ON players.player_id = lastplay.player_lastday) as days_info) as player_status

ON player_status.player_id = player_spent.pid) as table1

GROUP BY 1
ORDER BY 2 DESC

#### *This shows the retained players have spent more than the unretained players*

---

# **Do players with rolling 30-day retention come from specific regions?**

#### 10. Using the query in In[12] filtering the data for retained players and joining it with player info table to get location specific retention data for players.

In [ ]:
%%bigquery
SELECT location,
    COUNT(player_id) as total_retained_players
FROM
(
SELECT days_info.joining_day,
        days_info.player_id as pid
    
FROM
(SELECT players.player_id,
    players.joined as joining_day,
    lastplay.lastplay_day
FROM `junobigqueryclass.game_data.player_info` players
JOIN 
(
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1
) as lastplay

ON players.player_id = lastplay.player_lastday) as days_info

WHERE days_info.lastplay_day - days_info.joining_day > 30
) AS r_info

JOIN `junobigqueryclass.game_data.player_info` as p_info

ON p_info.player_id = r_info.pid
GROUP BY 1
ORDER BY 2 DESC

###  *Most Retained players came from South America*